# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

In [ ]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  h = 1e-10
  d1 = f(a, b, c)

  # partial derivative with respect to a
  d2 = f(a + h, b, c)
  df_da = (d2 - d1) / h

  # partial derivative with respect to b
  d2 = f(a, b + h, c)
  df_db = (d2 - d1) / h

  # partial derivative with respect to c
  d2 = f(a, b, c + h)
  df_dc = (d2 - d1) / h

  return [df_da, df_db, df_dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.35355817641448
OK for dim 1: expected 10.25699027111255, yours returns 10.256986371359744
OK for dim 2: expected 0.0625, yours returns 0.06250111539429781


In [ ]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
h = 1e-10
y1 = f(2, 3 ,4)

y2 = f(2 + h, 3, 4)
df_da = (y2 - y1) / h

y2 = f(2, 3 + h, 4)
df_db = (y2 - y1) / h

y2 = f(2, 3, 4 + h)
df_dc = (y2 - y1) / h

numerical_grad = [df_da, df_db, df_dc]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.35355817641448
OK for dim 1: expected 10.25699027111255, yours returns 10.256986371359744
OK for dim 2: expected 0.0625, yours returns 0.06250111539429781


In [ ]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
h = 1e-10

y1 = f(2 - h, 3, 4)
y2 = f(2 + h, 3, 4)
df_da = (y2 - y1) / (2*h)

y1 = f(2, 3 - h, 4)
y2 = f(2, 3 + h, 4)
df_db = (y2 - y1) / (2*h)

y1 = f(2, 3, 4 - h)
y2 = f(2, 3, 4 + h)
df_dc = (y2 - y1) / (2*h)

numerical_grad2 = [df_da, df_db, df_dc]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

# confirm that for the same step size h this version gives a
# better approximation.

for dim in range(3):
  sid = abs(numerical_grad[dim] - ans[dim]) # simple derivative result
  syd = abs(numerical_grad2[dim] - ans[dim]) # symmetric derivative result

  print(f"\nDim {dim}:\nSimple derivative: %.10f | Symmetric derivative: %.10f" % (sid, syd))
  if sid >= syd:
    print("Symmetric derivative has better approximation!")
  else:
    print("Simple derivative has better approximation!")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553735522382
OK for dim 1: expected 10.25699027111255, yours returns 10.256990812251843
OK for dim 2: expected 0.0625, yours returns 0.06250111539429781

Dim 0:
Simple derivative: 0.0000047858 | Symmetric derivative: 0.0000003449
Symmetric derivative has better approximation!

Dim 1:
Simple derivative: 0.0000038998 | Symmetric derivative: 0.0000005411
Symmetric derivative has better approximation!

Dim 2:
Simple derivative: 0.0000011154 | Symmetric derivative: 0.0000011154
Symmetric derivative has better approximation!


## section 2: support for softmax

In [ ]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __radd__(self, other):
    if isinstance(other, (int, float)):
      return self + other
    else:
      return other + self

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out

  def __rmul__(self, other):
    return other * self

  def __pow__(self, other):
    assert isinstance(other, (int , float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
      self.grad += other * self.data**(other - 1) * out.grad
    out._backward = _backward

    return out

  def __truediv__(self, other):
    return self * other**-1

  def __neg__(self):
    return self * -1

  def __sub__(self, other):
    return self + (-other)

  def tanh(self):
    x = self.data
    t = (exp(2*x) - 1)/(exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward

    return out

  def exp(self):
    x = self.data
    out = Value(exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out

  def log(self):
    x = self.data
    out = Value(log(x), (self, ), 'log')

    def _backward():
      self.grad += (1 / x) * out.grad
    out._backward = _backward

    return out

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [ ]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [ ]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

# define params
_x1 = 2.0
_x2 = 0.0
_w1 = -3.0
_w2 = 1.0
_b = 6.8813735870195432

# get our code output by Value class
x1 = Value(_x1)
x2 = Value(_x2)
w1 = Value(_w1)
w2 = Value(_w2)
b = Value(_b)

n = x1*w1 + x2*w2 + b
o = n.tanh()

o.backward()

print("Our results:")
print(f"x1: {x1.grad}")
print(f"x2: {x2.grad}")
print(f"w1: {w1.grad}")
print(f"w2: {w2.grad}")

# get the results using torch
x1 = torch.Tensor([_x1]).double(); x1.requires_grad = True
x2 = torch.Tensor([_x2]).double(); x2.requires_grad = True
w1 = torch.Tensor([_w1]).double(); w1.requires_grad = True
w2 = torch.Tensor([_w2]).double(); w2.requires_grad = True
b = torch.Tensor([_b]).double();   b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

o.backward()

print("\nTorch results:")
print(f"x1: {x1.grad.item()}")
print(f"x2: {x2.grad.item()}")
print(f"w1: {w1.grad.item()}")
print(f"w2: {w2.grad.item()}")

Our results:
x1: -1.4999999999999996
x2: 0.4999999999999999
w1: 0.9999999999999998
w2: 0.0

Torch results:
x1: -1.5000003851533106
x2: 0.5000001283844369
w1: 1.0000002567688737
w2: 0.0
